JUPYTER 환경에서 진행하였습니다. 코드상의 파일 경로가 수정되어서 진행되었습니다.
====

In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import time
import cv2
import pickle
import torch
from sklearn.model_selection import GridSearchCV 
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from scipy.stats import mode
from sklearn.decomposition import PCA

In [2]:
arg_img_size = (128, 128)
arg_dense_sift = True
args_local_cluster = 200
args_global_cluster = 200
num_frame = 5
args_aggr = "bow"#"vlad" 
pca_vlad = 128

In [3]:
# train 비디오의 행동 분류 label read
root = "/kaggle/input/2021-ml-tp4/"
train_csv = pd.read_csv('train_label.csv')
train_csv_arr = np.asarray(train_csv)

# 데이터 셋에 존재하는 행동 분류 정보 read
classinfo = pd.read_csv("class_info.csv")
classinfo_arr = np.asarray(classinfo)


# train 비디오 경로
train_list = os.listdir("train")
train_list.sort()
train_list = [os.path.join("train", i) for i in train_list]

# test 비디오 경로
test_list = os.listdir("test")
test_list.sort()
test_list = [os.path.join("test", i) for i in test_list]

In [4]:
def video_to_frame(video_path, size, num_frame):
    
    #########################################################
    ## 비디오에서 프레임을 추출해주는 함수
    ## 
    ## Input 
    ##     video_path : 한 비디오의 경로
    ##     size : 비디오 내의 프레임을 읽을 때, 원하는 해상도 크기
    ##     num_frames : 한 비디오 내에서 읽을 프레임의 수
    ##
    ## Output
    ##     frames : 읽고 저장한 총 프레임
    #########################################################
    
    cap = cv2.VideoCapture(video_path)
    
    # 한 비디오의 총 프레임 수 반환 및 원하는 프레임 수 많큼 읽기 위해 읽을 프레임의 인덱스 설정
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sel_ind = np.linspace(0, total_frame-1, num_frame).astype("int")
    
    
    num=0
    frames = []
    for i in range(total_frame):
        
        # 읽을 프레임 인덱스의 경우 프레임 읽어 메모리에 저장, 아닐 경우 지나감
        if i in sel_ind:
            res, frame = cap.read()
            # 원하는 해상도로 조절 및 grayscale로 변환
            frame = cv2.resize(frame, size, interpolation = cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(frame)
        else:
            res = cap.grab()        
    cap.release()
    frames = np.asarray(frames)

    return frames

In [5]:
def computeSIFT(data, dense=False):
    
    #########################################################
    ## 비디오 내의 프레임에서 특징점(visual word)을 SIFT or DenseSIFT로 추출해주는 함수
    ## 
    ## Input 
    ##     data : 한 비디오에서 읽고 저장한 프레임
    ##     dense : SIFT or DenseSIFT 사용 여부
    ##
    ## Output
    ##     x : 프레임에 대해 추출된 특징점(visual word), dict 형태 -> x[0]이면 0번째 인덱스 프레임의 특징점(visual word) [n,128] 확인 가능
    #########################################################
    
    x = {}
    for i in range(0, len(data)):
        if dense:
            img = data[i]
            step_size = 8
            kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
            
            ####### Empty Module 1 : DenseSIFT ########
            # 기본 SIFT 와 동일하게 정의 
            # 기본 SIFT 에서는 detectAndCompute를 사용했지만 
            # Dense SIFT는 위에서 생성한 keypoint를 사용해 compute 만을 진행 
            ###########################################
            sift = cv2.SIFT_create() #SIFT와 동일하게 정의
            _, desc = sift.compute(img, kp) #츨력과정에서 descriptor만 필요하여서 다음과 같이 설정
        else:
            sift = cv2.SIFT_create()
            img = data[i]
            kp, desc = sift.detectAndCompute(img, None)
        x.update({i : desc})

    return x

In [6]:
# train 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 train_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
train_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(train_list, desc="Extract {} in train data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    train_local_desc.update({vid_path : local_desc})

# test 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 test_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
test_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(test_list, desc="Extract {} in test data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    test_local_desc.update({vid_path : local_desc})

Extract dsift in test data: 100%|████████████████████████████████████████████████████| 505/505 [00:15<00:00, 32.90it/s]


In [7]:
print("\n\nAggregate SIFT descriptor")
start = time.time()


# train 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 train_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 train_local_info에서 확인 가능
train_frame_total = []
train_local_desc_total = []
train_local_info = {}
for k, v in train_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            train_local_desc_total.extend(vv)
            train_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        train_local_info.update({k+", "+str(kk) : l_num})
train_local_desc_total = np.asarray(train_local_desc_total)
train_frame_total = np.asarray(train_frame_total)


# test 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 test_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 test_local_info에서 확인 가능
test_frame_total = []
test_local_desc_total = []
test_local_info = {}
for k, v in test_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            test_local_desc_total.extend(vv)
            test_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        test_local_info.update({k+", "+str(kk) : l_num})
test_local_desc_total = np.asarray(test_local_desc_total)
test_frame_total = np.asarray(test_frame_total)


print("\t{:3.2f}s\n\n".format(time.time()-start))



Aggregate SIFT descriptor
	9.86s




In [8]:
def clustering(train_desc, test_desc=None, n_clusters=200):
    #########################################################
    ## 모든 특징점들 중, 대표 특징점(codebook)을 선정하는 함수
    ## 
    ## Input 
    ##     train_desc : 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     test_desc : 모든 test 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     n_clusters : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     train_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     test_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     clusters : 대표 특징점(codebook)
    ##     kmeans : kmeans 인스턴스
    #########################################################
    
    
    
    ##### Empty Module 2 : 대표 특징점(codebook) 선정 ######
    # 제약 조건 : MiniBatchKMeans 사용, random_state=0 고정
    # sklearn의 MiniBatchKMeans를 활용하여 n_clusters 크기 만큼의 대표 특징점(codebook)을 선정
    ###########################################
    kmeans = MiniBatchKMeans(n_clusters = n_clusters, random_state = 0) #MininBatchKMeans 선언
    kmeans.fit(train_desc) #학습
    clusters = kmeans.cluster_centers_ #학습 결과 획득한 클러스터링한 중심을 가져옴
    train_pred = kmeans.predict(train_desc) #예측
    if test_desc is not None:
        test_pred = kmeans.predict(test_desc)
    else:
        test_pred = None
    return train_pred, test_pred, clusters, kmeans

In [9]:
# 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들로 대표 특징점(codebook)을 만들고,
# train 비디오의 모든 프레임에서 추출된 특징점(visual word)과 test 비디오의 모든 프레임에서 추출된 특징점(visual word)을 대표 특징점(codebook)에 할당
train_local_alloc, test_local_alloc, local_codebook, local_kmeans = clustering(train_local_desc_total, test_local_desc_total, args_local_cluster)

C:\Users\yoons\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


In [10]:
print(np.shape(train_local_desc_total)) #데이터 구조 확인 과정
print(np.shape(train_local_alloc))
print(np.shape(local_codebook))

(2585600, 128)
(2585600,)
(200, 128)


In [11]:
index = np.where(test_local_alloc == 1) #where() 작동 방식 체크 과정
list(index) #반환 값이 array 구조
print(len(index[0]))
print(index)
print(index[0][0])

3010
(array([  3894,   3910,   3926, ..., 645211, 645227, 645451], dtype=int64),)
3894


In [12]:
def VLAD(X, alloc, centers):
    #########################################################
    ## 이미지 feature인 VLAD feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     X : 한 프레임의 특징점(visual word)들
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     centers : 대표 특징점(codebook)
    ##
    ## Output
    ##     V : VLAD feature
    #########################################################
    
    m,d = X.shape
    k = centers.shape[0]
    
    # VLAD feature를 담기 위한 변수
    V=np.zeros([k,d])

    for i in range(k):
        if np.sum(alloc==i)>0:
            index = np.where(alloc == i) #where()로 조건에 맞는 값 확인
            for j in range(len(index[0])): #VLAD feature 계산을 위한 반복문
                V[i]= V[i] + X[index[0][j]] -centers[i] #계산식
            ####################### Empty Module 3 : VLAD ########################
            # 이미지에서 추출된 특징점(visual word) X와 이들이 대표 특징점(codebook)으로 할당된 정보 alloc을 이용해,
            # 동일한 대표 특징점(codebook)으로 할당된 특징점(visual word)들의 벡터 합 계산해 V[i]에 저장
            # hint : 바로 위 조건문의 조건 "alloc==i"의 의미를 파악하고 인덱싱에 활용
            ######################################################################
    
    # 후처리 과정
    V = V.flatten()
    V = np.sign(V)*np.sqrt(np.abs(V))
    if np.sqrt(np.dot(V,V))!=0:
        V = V/np.sqrt(np.dot(V,V))
    return V


def BoW(alloc, n_cluster):
    #########################################################
    ## 이미지 feature인 BoW feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     n_cluster : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     V : BoW feature
    #########################################################
    V, bins = np.histogram(alloc,bins=range(n_cluster)) #Bow feature 사용 과정
    ######################### Empty Module 4 : BoW ##########################
    # 이미지에서 추출된 특징점(visual word)이 대표 특징점(codebook)으로 할당된 정보 alloc의 histogram을 계산
    # np.histogram 함수 참고
    #########################################################################
    return V

In [13]:
print("\n\nAllocate center & Descript local histogram")
start = time.time()

# Train 비디오 내의 프레임 별로 이미지 feature 기술 -> train_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> train_global_desc_key
train_global_desc = []
train_global_desc_key = []
vi=0
for k, v in train_local_info.items():
    if v!=0:
        if args_aggr=="bow":            
            hist_desc = BoW(train_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(train_local_desc_total[vi:vi+v], train_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        train_global_desc.append(hist_desc)
        train_global_desc_key.append(k)
train_global_desc = np.asarray(train_global_desc)
train_global_desc_key = np.asarray(train_global_desc_key)


# Test 비디오 내의 프레임 별로 이미지 feature 기술 -> test_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> test_global_desc_key
test_global_desc = []
test_global_desc_key = []
vi=0
for k, v in test_local_info.items():
    if v!=0:
        if args_aggr=="bow":
            hist_desc = BoW(test_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(test_local_desc_total[vi:vi+v], test_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        test_global_desc.append(hist_desc)
        test_global_desc_key.append(k)
test_global_desc = np.asarray(test_global_desc)
test_global_desc_key = np.asarray(test_global_desc_key)

print("\t{:3.2f}s\n\n".format(time.time()-start))



Allocate center & Descript local histogram
	1.59s




In [14]:
np.shape(train_global_desc) #데이터 구조 확인

(10100, 199)

In [15]:
np.shape(train_global_desc_key) #데이터 구조 확인

(10100,)

In [16]:
# VLAD feature의 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_global_desc)
    train_global_desc = pca.transform(train_global_desc)
    test_global_desc = pca.transform(test_global_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))

In [17]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 train 비디오의 각 프레임 별 label 가공
train_global_id = np.array([int(i.split("\\")[-1].split(".")[0]) for i in train_global_desc_key])
train_global_label = []
for fid in train_global_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_global_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_global_label = np.asarray(train_global_label).ravel()

# 분류를 위해, 행동 분류에 대한 test 비디오의 각 프레임 별 id 가공 
test_global_id = np.array([int(i.split("\\")[-1].split(".")[0]) for i in test_global_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))



Processing label
	0.49s




In [18]:
def saveFile(predict, predict_id, name, best_params=None):
    #########################################################
    ## 결과를 저장하기 위한 함수
    ## 
    ## Input 
    ##     predict : 모든 test 비디오의 행동 예측 값
    ##     predict_id : 모든 test 비디오의 id
    ##     name : 원하는 저장 파일 이름
    ##     best_params : 원하는 instance 저장 시 사용
    ##
    #########################################################
    
    data = np.concatenate((np.expand_dims(predict_id.astype("str"), axis=1), np.expand_dims(predict.astype("str"), axis=1)), axis=1)
    csv = pd.DataFrame(data, columns=['Id', 'Category'])
    csv.to_csv(name + ".csv", index=False)
    
    if best_params:
        f = open(name + ".pickle", "wb")
        pickle.dump(best_params, f, 2)

In [19]:
test_global_id #데이터 확인

array([  0,   0,   0, ..., 504, 504, 504])

In [20]:
np.shape(train_global_desc) #데이터 구조 확인

(10100, 199)

In [21]:
np.unique(train_global_id) #데이터 확인

array([   0,    1,    2, ..., 2017, 2018, 2019])

In [22]:
print("\n\nSVM global averaging in frame")
start = time.time()
#################### Empty Module 6 : SVM (averaging) ######################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 비디오 별로 평균 내어 비디오 feature로 사용
# 비디오 feature로 학습 후, 행동 예측
# hint : 위 셀에서 선언한 train_global_id, train_global_label, test_global_id 활용
###########################################################################
parameters = {'kernel':['rbf', 'poly', 'sigmoid'], 'C':[0.01, 0.1 ,1, 10, 100, 1000]} #파라미터 설정
model=GridSearchCV(SVC(kernel = 'linear'),parameters,cv=5) #GridSearchCV 사용
train_global_desc_average = [] #리스트 선언
train_global_label_average = []
test_global_desc_average = []
np.array(train_global_desc_average)#구조 선언
np.array(train_global_label_average)
np.array(test_global_desc_average)

for i in range(2019):
    train_global_desc_average.append(np.mean(train_global_desc[5*i : 5*i+5], axis = 0)) #average feature 사용을 위한 계산식
    train_global_label_average.append(np.mean(train_global_label[5*i : 5*i+5], axis = 0).astype(int)) #average 후 데이터 타입 선언
for i in range(505):
    test_global_desc_average.append(np.mean(test_global_desc[5*i : 5*i+5], axis = 0)) #average feature 사용을 위한 계산식
#print(np.shape(train_global_desc_average))
#print(np.shape(train_global_label_average))
#print(np.shape(test_global_desc_average))
model.fit(train_global_desc_average, train_global_label_average) #학습
svm_predict = model.predict(test_global_desc_average) #예측
#print(np.shape(svm_predict))
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_averaging")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global averaging in frame
	66.99s




In [23]:
print("\n\nSVM global voting in frame")
start = time.time()
############################### Empty Module 5 : SVM (voting) ##################################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 모두 사용해 SVM 학습
# 학습 시, 각 이미지 feature의 label은 해당되는 비디오의 label(행동)로 사용
# 프레임 별로 행동 예측 후, 같은 비디오 내 프레임의 행동 예측 값의 최빈값을 해당 비디오의 행동 예측 값으로 선정
# hint : 위 셀에서 선언한 train_global_label, test_global_id 및 mode 활용
################################################################################################
model.fit(train_global_desc, train_global_label)
svm_predict = []
test_predict = model.predict(test_global_desc)
#print(np.shape(test_predict))
for i in range(505):
    #print((mode(test_predict[5*i : 5*i+5])[0])[0])
    svm_predict.append((mode(test_predict[5*i : 5*i+5])[0])[0]) #mode를 활용하여 voting 결과 추출

#print(np.shape(svm_predict))
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_voting")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global voting in frame
	1164.26s




In [24]:
train_global_alloc, test_global_alloc, global_codebook, global_kmeans = clustering(train_global_desc, test_global_desc, args_global_cluster)

C:\Users\yoons\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


In [25]:
print("\n\nAllocate center & Descript global histogram")
start = time.time()
train_vid_names = np.asarray([i.split(", ")[0] for i in train_global_desc_key])
train_vid_names_u = np.unique(train_vid_names)

# Train 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
train_video_desc = []
train_video_desc_key = []
for vid_name in train_vid_names_u:
    cind = np.where(vid_name==train_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(train_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(train_global_desc[cind], train_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    train_video_desc.append(hist_desc)
    train_video_desc_key.append(vid_name)
train_video_desc = np.asarray(train_video_desc)
train_video_desc_key = np.asarray(train_video_desc_key)

# Test 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
test_vid_names = np.asarray([i.split(", ")[0] for i in test_global_desc_key])
test_vid_names_u = np.unique(test_vid_names)

test_video_desc = []
test_video_desc_key = []
for vid_name in test_vid_names_u:
    cind = np.where(vid_name==test_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(test_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(test_global_desc[cind], test_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    test_video_desc.append(hist_desc)
    test_video_desc_key.append(vid_name)
test_video_desc = np.asarray(test_video_desc)
test_video_desc_key = np.asarray(test_video_desc_key)


print("\t{:3.2f}s\n\n".format(time.time()-start))



Allocate center & Descript global histogram
	0.51s




In [26]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 각 train 비디오 별 label 가공
train_video_id = np.array([int(i.split("\\")[-1].split(".")[0]) for i in train_video_desc_key])
train_video_label = []
for fid in train_video_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_video_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_video_label = np.asarray(train_video_label).ravel()

# 분류를 위해, 행동 분류에 대한 각 test 비디오 별 id 가공
test_video_id = np.array([int(i.split("\\")[-1].split(".")[0]) for i in test_video_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))



Processing label
	0.09s




In [27]:
# 이미지 feature에 대해 다시 한번 VLAD feature 기술 방식을 사용하여 video feature를 기술한 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_video_desc)
    train_video_desc = pca.transform(train_video_desc)
    test_video_desc = pca.transform(test_video_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))

In [28]:
np.shape(train_video_desc)

(2020, 199)

In [29]:
np.shape(train_video_label)

(2020,)

In [30]:
print("\n\nSVM video descriptor")
start = time.time()
######################## Empty Module 7 : SVM (video feature) ##########################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 이전 이미지 feature를 기반으로 각 기술방식(BoW, VLAD)을 한번 더 적용해 만든 비디오 feature 사용
# 비디오 feature로 학습 후, 행동 예측
#######################################################################################
model.fit(train_video_desc, train_video_label) #기존에 선언했던 모델 사용
svm_predict = model.predict(test_video_desc) #예측
saveFile(classinfo_arr[svm_predict][:,1], test_video_id, "svm_video")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM video descriptor
	53.19s


